In [2]:
import requests
import pandas as pd
from pprint import pprint
import time
import json
import csv
import math
import numpy

In [3]:
cdc500 = pd.read_csv("500_Cities__Local_Data_for_Better_Health__2018_release.csv", encoding="utf-8")
cdc5002=cdc500[(cdc500['GeographicLevel'] == 'City')&(cdc500['DataValueTypeID'] == 'AgeAdjPrv')&(cdc500['Measure'] == "Visits to doctor for routine checkup within the past Year among adults aged >=18 Years")]
cdc5002['PerYrlyDoc']=cdc5002['Data_Value']
cdcdoc = cdc5002[['StateAbbr','CityName','PerYrlyDoc']]


cdcdoc.head(5)


C:\Users\silva\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,StateAbbr,CityName,PerYrlyDoc
860,AL,Birmingham,76.9
3099,AL,Hoover,72.7
4246,AL,Huntsville,69.1
6191,AL,Mobile,74.1
8285,AL,Montgomery,76.7


In [4]:
#summarize by month by city, then average# of international arrivals by month by city

fldf= pd.read_csv("US International Flight Arrival 2018.csv", encoding="utf-8",usecols=['PASSENGERS','DEST','DEST_CITY_NAME','YEAR','MONTH'])
fldf[['City','State']] = fldf['DEST_CITY_NAME'].str.split(', ',expand=True)
fl= fldf.groupby(['City','State'])[["PASSENGERS"]].sum()
fl

,,PASSENGERS
City,State,
Abilene,TX,6
Aguadilla,PR,905
Akron,OH,21
Alamogordo,NM,0
Albany,NY,342
Albuquerque,NM,672
Alexandria,LA,2714
Allentown/Bethlehem/Easton,PA,7
Altoona,PA,1


In [5]:
cdcp=pd.read_csv("NationalOutbreakPersonToPerson.csv", encoding="utf-8")
cdcp=cdcp[(cdcp['Etiology Status'] == 'Confirmed')&(cdcp['Year'] == 2017)]
cdcpr= cdcp.groupby(['State'])[["Illnesses"]].sum()

cdcpr


,Illnesses
State,
Alabama,688
Alaska,126
Arizona,239
Colorado,54
Connecticut,86
Florida,32
Georgia,25
Hawaii,48
Illinois,3191


In [6]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}
states=pd.DataFrame.from_dict(us_state_abbrev,orient='index', columns=['abrev'])
states.reset_index(level=0, inplace=True)
states=states.rename(columns={states.columns[0]:'name'})
states.head()

,name,abrev
0,Mississippi,MS
1,Oklahoma,OK
2,Delaware,DE
3,Minnesota,MN
4,Illinois,IL


In [7]:
sub1 = pd.merge(cdcpr, states,  how='left', left_on=['State'], right_on = ['name'])
sub1

,Illnesses,name,abrev
0,688,Alabama,AL
1,126,Alaska,AK
2,239,Arizona,AZ
3,54,Colorado,CO
4,86,Connecticut,CT
5,32,Florida,FL
6,25,Georgia,GA
7,48,Hawaii,HI
8,3191,Illinois,IL
9,1247,Indiana,IN


In [8]:
sub2 = pd.merge(cdcdoc, sub1,  how='left', left_on=['StateAbbr'], right_on = ['abrev'])
sub2 = sub2.drop(columns=["name", "abrev"])

sub2.head()

,StateAbbr,CityName,PerYrlyDoc,Illnesses
0,AL,Birmingham,76.9,688.0
1,AL,Hoover,72.7,688.0
2,AL,Huntsville,69.1,688.0
3,AL,Mobile,74.1,688.0
4,AL,Montgomery,76.7,688.0


In [9]:
sub3 = pd.merge(sub2, fl,  how='left', left_on=['StateAbbr','CityName'], right_on = ['State','City'])
sub3.sort_values(by='PASSENGERS', ascending=False)
sub3.head()



,StateAbbr,CityName,PerYrlyDoc,Illnesses,PASSENGERS
0,AL,Birmingham,76.9,688.0,971.0
1,AL,Hoover,72.7,688.0,NaN
2,AL,Huntsville,69.1,688.0,19.0
3,AL,Mobile,74.1,688.0,NaN
4,AL,Montgomery,76.7,688.0,NaN


In [10]:
citypop= pd.read_csv("popcsv.csv", encoding="utf-8",usecols=['city','region','population'])
citypop['city']=citypop['city'].str.title()
citypop.head()

,city,population,region
0,Lynden,10816.0,WA
1,Ferndale,10098.0,WA
2,Bellingham,73469.0,WA
3,Havre,9379.0,MT
4,Anacortes,16061.0,WA


In [11]:
sub4 = pd.merge(sub3, citypop,  how='left', left_on=['StateAbbr','CityName'], right_on = ['region','city'])

sub4['perNoYrly']=100-sub4['PerYrlyDoc']
sub4['Ycity']=sub4['CityName']+", "+sub4['StateAbbr']
sub4 = sub4.drop(columns=["city", "region",'PerYrlyDoc'])
sub4.sort_values(by='population', ascending=False)
#sub4=sub4.dropna(subset=['PASSENGERS'])
sub4=sub4.fillna(0)

sub4.head()


,StateAbbr,CityName,Illnesses,PASSENGERS,population,perNoYrly,Ycity
0,AL,Birmingham,688.0,971.0,231621.0,23.1,"Birmingham, AL"
1,AL,Hoover,688.0,0.0,66752.0,27.3,"Hoover, AL"
2,AL,Huntsville,688.0,19.0,167528.0,30.9,"Huntsville, AL"
3,AL,Mobile,688.0,0.0,190274.0,25.9,"Mobile, AL"
4,AL,Montgomery,688.0,0.0,198325.0,23.3,"Montgomery, AL"


In [31]:
den=pd.read_csv("popden.csv", encoding="utf-8")
den['City'] = den['City'].str.strip()
den.head()

,Unnamed: 0,DENSITY,POP,City,State,abrev
0,7349,421.729903,1924,Zwolle,Louisiana,LA
1,6002,570.717135,93,Zwingle,Iowa,IA
2,6629,527.183919,95,Zurich,Kansas,KS
3,8969,1265.090805,3455,Zumbrota,Minnesota,MN
4,8968,437.118017,240,Zumbro Falls,Minnesota,MN


In [40]:
sub5 = pd.merge(sub4, den,  how='left', left_on=['StateAbbr','CityName'], right_on = ['abrev','City'])
sub5 = sub5[sub5.StateAbbr != 'HI']
sub5 = sub5[sub5.StateAbbr != 'AK']
sub5 = sub5.drop(columns=['StateAbbr','CityName',"City", 'StateAbbr','CityName','abrev','State'])
sub5=sub5.fillna(0)

sub5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 499
Data columns (total 8 columns):
Illnesses     498 non-null float64
PASSENGERS    498 non-null float64
population    498 non-null float64
perNoYrly     498 non-null float64
Ycity         498 non-null object
Unnamed: 0    498 non-null float64
DENSITY       498 non-null float64
POP           498 non-null float64
dtypes: float64(7), object(1)
memory usage: 35.0+ KB


In [41]:
sub6 = sub5[sub5.POP != 0]
sub6=sub6.sort_values(by='POP')
sub6.head(20)

,Illnesses,PASSENGERS,population,perNoYrly,Ycity,Unnamed: 0,DENSITY,POP
465,35.0,496.0,38601.0,34.4,"Burlington, VT",18017.0,4162.364871,42899.0
491,368.0,0.0,50427.0,20.8,"Charleston, WV",18604.0,1498.845991,47215.0
499,276.0,0.0,55443.0,36.6,"Cheyenne, WY",19410.0,2244.368331,63957.0
370,625.0,0.0,77540.0,23.7,"Youngstown, OH",13762.0,1914.416090,64958.0
344,132.0,0.0,60280.0,26.5,"Schenectady, NY",11813.0,6076.186607,65575.0
266,338.0,11.0,63142.0,29.7,"Portland, ME",7365.0,3079.285585,66417.0
261,508.0,0.0,69721.0,28.1,"Kenner, LA",7192.0,4479.519520,66657.0
101,0.0,0.0,67791.0,32.8,"Redondo Beach, CA",1543.0,10877.264583,67412.0
340,132.0,0.0,68137.0,24.4,"Mount Vernon, NY",11678.0,15417.560565,67593.0
251,453.0,0.0,65836.0,30.3,"Waterloo, IA",5947.0,1102.564796,67798.0


In [42]:
sub6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 465 to 342
Data columns (total 8 columns):
Illnesses     497 non-null float64
PASSENGERS    497 non-null float64
population    497 non-null float64
perNoYrly     497 non-null float64
Ycity         497 non-null object
Unnamed: 0    497 non-null float64
DENSITY       497 non-null float64
POP           497 non-null float64
dtypes: float64(7), object(1)
memory usage: 34.9+ KB


In [43]:
sub6.to_csv("datatotest.csv", encoding="utf-8" )

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [87]:
X_train.head()

,Illnesses,PASSENGERS,POP,perNoYrly,DENSITY
17,0.0,109065.0,181107.0,35.1,3624.828376
113,1093.0,101575.0,179335.0,18.9,9741.908813
143,34.0,28.0,219190.0,37.5,3187.438998
69,174.0,2.0,185877.0,22.0,4975.346092
75,203.0,1493924.0,425403.0,30.8,7877.719104


In [88]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [89]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train.ravel())


rf.score(X_test_scaled, y_test)

0.0

In [82]:
feature_names=X.columns
importantDict=sorted(zip(rf.feature_importances_, feature_names), reverse=True)
importantDict

[(0.26244628154918376, u'POP'),
 (0.26127091763697696, u'DENSITY'),
 (0.2599693556386215, 'perNoYrly'),
 (0.13540928330728008, 'Illnesses'),
 (0.08090416186793763, 'PASSENGERS')]